# PyOpenCL

In [1]:
import pyopencl as cl
import pyopencl.array
import pyopencl.clrandom
import numpy as np

In [2]:
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)

## Prefix Sum

In [3]:
from pyopencl.scan import GenericScanKernel

Cumulative sum of squares of elements of x

In [4]:
x = cl.clrandom.rand(queue, 10**7, dtype=np.float64)

y = cl.array.cumsum(x * x).get()

/Users/aditya/miniconda2/lib/python2.7/site-packages/pyopencl/cffi_cl.py:1502: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  "to see more.", CompilerWarning)


In [5]:
print(y)

[  6.37068889e-01   1.22605937e+00   1.70115412e+00 ...,   3.33233516e+06
   3.33233525e+06   3.33233532e+06]


`GenericScanKernel(context, dtype, arguments, input_expr, scan_expr, neutral, output_statement)`

In [6]:
knl = GenericScanKernel(ctx, np.float64,
    arguments="__global double* x",
    input_expr="x[i]*x[i]",
    scan_expr="a+b", neutral="0",
    output_statement="x[i] = item")

In [7]:
knl(x)

In [8]:
print(x)

[  6.37068889e-01   1.22605937e+00   1.70115412e+00 ...,   3.33233516e+06
   3.33233525e+06   3.33233532e+06]


Copy elements greater than 300 from an array a to array b ([source](https://documen.tician.de/pyopencl/algorithm.html#usage-example))

In [9]:
 knl = GenericScanKernel(
        ctx, np.int32,
        arguments="__global int* ary, __global int* out",
        input_expr="(ary[i] > 300) ? 1 : 0",
        scan_expr="a+b", neutral="0",
        output_statement="""
            if (prev_item != item) out[item-1] = ary[i];
            """)

In [10]:
a = cl.array.arange(queue, 250, 350, 1, dtype=np.int32)
out = cl.array.empty_like(a)
print(a)

[250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267
 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285
 286 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303
 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321
 322 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339
 340 341 342 343 344 345 346 347 348 349]


In [11]:
knl(a, out)

In [12]:
print(out)

[301 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318
 319 320 321 322 323 324 325 326 327 328 329 330 331 332 333 334 335 336
 337 338 339 340 341 342 343 344 345 346 347 348 349   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]
